# Heat Exchanger

Story/Vorturnen

- Adiabaten Wärmetauscher, der Wärme vollständig von Fluid a auf Fluid b transportiert
- Wo ist da ein Energieverlust?
- Beispiel mit Temperaturen/Temperaturniveaus, es ist nicht umkehrbar!
- Exergieanalyse zeigt auch hier Qualitätsverlust
- 1 kleines Beispiel für die Berechnung + Plots der Exergieverluste

Übung

- Berechnung von Fuel/Product/Destruction/Wirkungsgrad (Verweis auf Grundlagen) mit allen T > T0
- Änderung von T0 aber immernoch T > T0 analysieren, Parameteranalyse?
- Wie spielen Druckverluste eine Rolle?
- Kondensation/Verdampfung?
- Wärmeübetragung vollständig unter T0 berechnen, hierzu Story erzählen, was Nutzen und Aufwand sind.

## Introduction

A heat exchanger is used to transfer heat from one fluid to a different one. We can write the energy balance for the component assuming, that no heat is lost to the ambient.

```{math}
    :label: heat-exchanger-energy-balance
    0 = \dot m_1 \cdot \left(h_\text{1,out} - h_\text{1,in}\right) + \dot m_2 \cdot \left(h_\text{2,out} - h_\text{2,in}\right)
```

The hot side {math}`1` transfers heat to the cold side {math}`2`. We can see, that all heat provided by the hot side is transferred to the cold side. However, in this process it is not possible to reach the same temperature level on the cold side as on the hot side. Reversing the heat transfer is for the same reason not possible: You cannot transfer heat against the direction of the temperature gradient. That obviously means, that the heat transfer incurs thermodynamic losses. To make them visible, we will again calculate the exergy of each of the flows.

As an example we want to have a look at a home heating system that uses centralized distric heat as a source. To separate the district heating system from the home system, a heat exchangers is set in place. The radiator of the home system needs a feed flow temperature of {math}`70` degrees Celsius. By transfering heat to the home, its temperature reduces by {math}`15` Kelvin. The district heating system provides a flow of {math}`5 \frac{kg}{s}` with a temperature of {math}`90` degrees Celsius. It returns back with a temperature of {math}`50` degrees Celsius.


In [1]:
from CoolProp.CoolProp import PropsSI as PSI

# Properties of hot side fluid
fluid_1 = 'water'
m_dot_1 = 1  # kg/s
p_1 = (1.013 + 8) * 1e5  # Pa
T_in_1 = 90 + 273.15  # K
T_out_1 = 50 + 273.15  # K

# Properties of cold side fluid
fluid_2 = 'water'
p_2 = 1.013 * 1e5  # Pa
T_in_2 = 55 + 273.15  # K
T_out_2 = 70 + 273.15  # K


Let's find out how much water is flowing through the home heating system. To employ equation {eq}`heat-exchanger-energy-balance`, we have to get the specific enthalpies of our fluids.

In [2]:
h_in_1 = PSI('H', 'P', p_1, 'T', T_in_1, fluid_1)
h_out_1 = PSI('H', 'P', p_1, 'T', T_out_1, fluid_1)

h_in_2 = PSI('H', 'P', p_2, 'T', T_in_2, fluid_2)
h_out_2 = PSI('H', 'P', p_2, 'T', T_out_2, fluid_2)

m_dot_2 = m_dot_1 * (h_out_1 - h_in_1) / -(h_out_2 - h_in_2)
m_dot_2

2.668658947365666


```{tip}
We have provided the function to calculate the exergy in a scripts folder, so we can import it from there instead of redefining it here.
```

In [6]:
# from scripts.utilities import calc_physical_exergy


# ex_in_1
# ex_in_2
# ex_out_2
# ex_out_3

ModuleNotFoundError: No module named 'scripts.utilities'

In [ ]:
# from CoolProp.CoolProp import PropsSI as PSI

# # Properties of hot side fluid
# fluid_1 = 'air'
# m_dot_1 = 10  # kg/s
# p_1 = 1.013 * 1e5  # Pa
# T_in_1 = 100 + 273.15  # K

# # Properties of cold side fluid
# fluid_2 = 'air'
# m_dot_1 = 9  # kg/s
# p_2 = 1.013 * 1e5  # Pa
# T_in_2 = 10 + 273.15  # K
# dT_terminal = 5  # K


## Lessons Learned

- What did you learn?
- What should you be able to apply yourself?